In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [106]:
class TwoLayerNet(object):
  """
  A two-layer fully-connected neural network. The net has an input dimension of
  N, a hidden layer dimension of H, and performs classification over C classes.
  We train the network with a softmax loss function and L2 regularization on the
  weight matrices. The network uses a ReLU nonlinearity after the first fully
  connected layer.
  In other words, the network has the following architecture:
  input - fully connected layer - ReLU - fully connected layer - softmax
  The outputs of the second fully-connected layer are the scores for each class.
  """

  def __init__(self, input_size, hidden_size, output_size, std=1e-4):
    """
    Initialize the model. Weights are initialized to small random values and
    biases are initialized to zero. Weights and biases are stored in the
    variable self.params, which is a dictionary with the following keys:
    W1: First layer weights; has shape (D, H)
    b1: First layer biases; has shape (H,)
    W2: Second layer weights; has shape (H, C)
    b2: Second layer biases; has shape (C,)
    Inputs:
    - input_size: The dimension D of the input data.
    - hidden_size: The number of neurons H in the hidden layer.
    - output_size: The number of classes C.
    """
    self.params = {}
    self.params['W1'] = std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

  def loss(self, X, y=None, reg=0.0):
    """
    Compute the loss and gradients for a two layer fully connected neural
    network.
    Inputs:
    - X: Input data of shape (N, D). Each X[i] is a training sample.
    - y: Vector of training labels. y[i] is the label for X[i], and each y[i] is
      an integer in the range 0 <= y[i] < C. This parameter is optional; if it
      is not passed then we only return scores, and if it is passed then we
      instead return the loss and gradients.
    - reg: Regularization strength.
    Returns:
    If y is None, return a matrix scores of shape (N, C) where scores[i, c] is
    the score for class c on input X[i].
    If y is not None, instead return a tuple of:
    - loss: Loss (data loss and regularization loss) for this batch of training
      samples.
    - grads: Dictionary mapping parameter names to gradients of those parameters
      with respect to the loss function; has the same keys as self.params.
    """
    # Unpack variables from the params dictionary
    W1, b1 = self.params['W1'], self.params['b1']
    W2, b2 = self.params['W2'], self.params['b2']
    N, D = X.shape

    # Compute the forward pass
    scores = None
    #############################################################################
    # TODO: Perform the forward pass, computing the class scores for the input. #
    # Store the result in the scores variable, which should be an array of      #
    # shape (N, C).                                                             #
    #############################################################################
    h_output = np.maximum(0, X.dot(W1) + b1) #(N,D) * (D,H) = (N,H)
    print("h_output",h_output.shape)
    scores = h_output.dot(W2) + b2
    print("scores", scores.shape)
    # pass
    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################
    
    # If the targets are not given then jump out, we're done
    if y is None:
      return scores

    # Compute the loss
    loss = None
    #############################################################################
    # TODO: Finish the forward pass, and compute the loss. This should include  #
    # both the data loss and L2 regularization for W1 and W2. Store the result  #
    # in the variable loss, which should be a scalar. Use the Softmax           #
    # classifier loss. So that your results match ours, multiply the            #
    # regularization loss by 0.5                                                #
    #############################################################################
    shift_scores = scores - np.max(scores, axis = 1).reshape(-1,1)
    print "shift_scores:",shift_scores.shape, shift_scores
    softmax_output = np.exp(shift_scores)/np.sum(np.exp(shift_scores), axis = 1).reshape(-1,1)
    print "softmax_output",softmax_output
    loss = -np.sum(np.log(softmax_output[range(N), list(y)]))
    loss /= N
    loss +=  0.5* reg * (np.sum(W1 * W1) + np.sum(W2 * W2))
    print("loss",loss)
    # pass
    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    # Backward pass: compute gradients
    grads = {}
    #############################################################################
    # TODO: Compute the backward pass, computing the derivatives of the weights #
    # and biases. Store the results in the grads dictionary. For example,       #
    # grads['W1'] should store the gradient on W1, and be a matrix of same size #
    #############################################################################
    # pass
    dscores = softmax_output.copy()
    dscores[range(N), list(y)] -= 1
    print "dscores",dscores
    dscores /= N
    grads['W2'] = h_output.T.dot(dscores) + reg * W2
    grads['b2'] = np.sum(dscores, axis = 0)
    
    dh = dscores.dot(W2.T)
    dh_ReLu = (h_output > 0) * dh
    grads['W1'] = X.T.dot(dh_ReLu) + reg * W1
    grads['b1'] = np.sum(dh_ReLu, axis = 0)
    
    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    return loss, grads

  def train(self, X, y, X_val, y_val,
            learning_rate=1e-3, learning_rate_decay=0.95,
            reg=1e-5, num_iters=100,
            batch_size=200, verbose=False):
    """
    Train this neural network using stochastic gradient descent.
    Inputs:
    - X: A numpy array of shape (N, D) giving training data.
    - y: A numpy array f shape (N,) giving training labels; y[i] = c means that
      X[i] has label c, where 0 <= c < C.
    - X_val: A numpy array of shape (N_val, D) giving validation data.
    - y_val: A numpy array of shape (N_val,) giving validation labels.
    - learning_rate: Scalar giving learning rate for optimization.
    - learning_rate_decay: Scalar giving factor used to decay the learning rate
      after each epoch.
    - reg: Scalar giving regularization strength.
    - num_iters: Number of steps to take when optimizing.
    - batch_size: Number of training examples to use per step.
    - verbose: boolean; if true print progress during optimization.
    """
    num_train = X.shape[0]
    iterations_per_epoch = max(num_train / batch_size, 1)

    # Use SGD to optimize the parameters in self.model
    loss_history = []
    train_acc_history = []
    val_acc_history = []

    for it in xrange(num_iters):
      X_batch = None
      y_batch = None

      #########################################################################
      # TODO: Create a random minibatch of training data and labels, storing  #
      # them in X_batch and y_batch respectively.                             #
      #########################################################################
      idx = np.random.choice(num_train, batch_size, replace=True)
      X_batch = X[idx]
      y_batch = y[idx]
      # pass
      #########################################################################
      #                             END OF YOUR CODE                          #
      #########################################################################

      # Compute loss and gradients using the current minibatch
      loss, grads = self.loss(X_batch, y=y_batch, reg=reg)
      loss_history.append(loss)

      #########################################################################
      # TODO: Use the gradients in the grads dictionary to update the         #
      # parameters of the network (stored in the dictionary self.params)      #
      # using stochastic gradient descent. You'll need to use the gradients   #
      # stored in the grads dictionary defined above.                         #
      #########################################################################
      self.params['W2'] += - learning_rate * grads['W2']
      self.params['b2'] += - learning_rate * grads['b2']
      self.params['W1'] += - learning_rate * grads['W1']
      self.params['b1'] += - learning_rate * grads['b1']
      # pass
      #########################################################################
      #                             END OF YOUR CODE                          #
      #########################################################################

      if verbose and it % 100 == 0:
        print 'iteration %d / %d: loss %f' % (it, num_iters, loss)

      # Every epoch, check train and val accuracy and decay learning rate.
      if it % iterations_per_epoch == 0:
        # Check accuracy
        train_acc = (self.predict(X_batch) == y_batch).mean()
        val_acc = (self.predict(X_val) == y_val).mean()
        train_acc_history.append(train_acc)
        val_acc_history.append(val_acc)

        # Decay learning rate
        learning_rate *= learning_rate_decay

    return {
      'loss_history': loss_history,
      'train_acc_history': train_acc_history,
      'val_acc_history': val_acc_history,
    }

  def predict(self, X):
    """
    Use the trained weights of this two-layer network to predict labels for
    data points. For each data point we predict scores for each of the C
    classes, and assign each data point to the class with the highest score.
    Inputs:
    - X: A numpy array of shape (N, D) giving N D-dimensional data points to
      classify.
    Returns:
    - y_pred: A numpy array of shape (N,) giving predicted labels for each of
      the elements of X. For all i, y_pred[i] = c means that X[i] is predicted
      to have class c, where 0 <= c < C.
    """
    y_pred = None

    ###########################################################################
    # TODO: Implement this function; it should be VERY simple!                #
    ###########################################################################
    h = np.maximum(0, X.dot(self.params['W1']) + self.params['b1'])
    scores = h.dot(self.params['W2']) + self.params['b2']
    y_pred = np.argmax(scores, axis=1)
    # pass
    ###########################################################################
    #                              END OF YOUR CODE                           #
    ###########################################################################

    return y_pred


In [107]:
tl=TwoLayerNet(10,8,3)

In [108]:
tl.params['W1'].shape

(10, 8)

In [109]:
tl.params['W2'].shape

(8, 3)

In [110]:
tl.params['b1'].shape

(8,)

In [111]:
tl.params['b2'].shape

(3,)

In [112]:
X=np.random.randn(100,10)

In [113]:
y=np.random.randint(3, size=100)

In [114]:
y

array([0, 0, 1, 1, 2, 2, 0, 1, 0, 1, 1, 0, 1, 0, 2, 2, 0, 0, 1, 1, 0, 2,
       2, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 0, 1, 2, 2, 1, 2, 0, 1,
       1, 1, 2, 2, 0, 1, 2, 0, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1,
       2, 2, 1, 0, 1, 1, 0, 0, 1, 0, 2, 1, 1, 1, 0, 2, 1, 1, 1, 2, 0, 2,
       0, 2, 0, 2, 1, 2, 1, 1, 2, 1, 1, 1])

In [115]:
tl.loss(X,y)

('h_output', (100, 8))
('scores', (100, 3))
shift_scores: (100, 3) [[-2.20605117e-08 -8.72239366e-08  0.00000000e+00]
 [ 0.00000000e+00 -2.15748614e-07 -5.62545151e-08]
 [-7.71148023e-08 -9.87536832e-08  0.00000000e+00]
 [-1.83632419e-08 -7.50145719e-08  0.00000000e+00]
 [ 0.00000000e+00 -9.57671488e-08 -1.34629267e-08]
 [-4.28170101e-08 -9.78811672e-09  0.00000000e+00]
 [-5.61381744e-10 -1.63726485e-07  0.00000000e+00]
 [ 0.00000000e+00 -1.51257263e-07 -2.59228395e-08]
 [-3.28381492e-08  0.00000000e+00 -2.25631777e-08]
 [ 0.00000000e+00 -4.04166900e-08 -2.64623962e-09]
 [-3.79671774e-08 -1.37372767e-07  0.00000000e+00]
 [ 0.00000000e+00 -1.95219091e-07 -3.17056308e-08]
 [-9.37269680e-09  0.00000000e+00 -1.06059367e-07]
 [-5.83571373e-09 -1.12524369e-08  0.00000000e+00]
 [ 0.00000000e+00 -5.49854384e-08 -1.35215367e-08]
 [-5.94114721e-09 -6.83962786e-08  0.00000000e+00]
 [ 0.00000000e+00 -6.86341834e-08 -4.82569158e-09]
 [-2.48044284e-08 -4.24280698e-08  0.00000000e+00]
 [-1.06932502e-

(1.0986122981630408,
 {'W1': array([[ 3.63416350e-06,  1.90500405e-06,  2.94076364e-07,
          -1.17497880e-06, -4.36599708e-06, -8.00811456e-07,
          -2.25240646e-06,  1.36351139e-06],
         [ 5.46958052e-06, -1.38084857e-06, -1.03789689e-05,
           4.06472026e-08,  2.35884680e-06,  1.21770002e-06,
           2.66631207e-06, -1.58554571e-06],
         [ 2.97805880e-06, -9.66781053e-07, -2.25949326e-06,
          -1.31107823e-06, -5.82026041e-06, -9.00008168e-07,
           6.32396686e-07,  4.95317189e-06],
         [-1.07735000e-05, -2.35928405e-07,  2.66871669e-06,
          -3.71902553e-08, -1.24311897e-05, -1.49225068e-06,
          -5.41433351e-06,  1.13381026e-06],
         [ 6.39217536e-06,  1.64338997e-06, -2.52236871e-06,
           1.36772474e-06,  3.45995553e-06, -8.80504643e-07,
           4.31118425e-06,  1.46435676e-06],
         [-1.59512758e-06,  2.27736822e-06,  6.84494357e-07,
           1.83850533e-06,  1.20757049e-05, -1.53089367e-06,
           5.222

In [119]:
range(10), list(y)

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0,
  0,
  1,
  1,
  2,
  2,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  2,
  2,
  0,
  0,
  1,
  1,
  0,
  2,
  2,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  2,
  0,
  1,
  2,
  2,
  1,
  2,
  0,
  1,
  1,
  1,
  2,
  2,
  0,
  1,
  2,
  0,
  2,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  1,
  1,
  2,
  2,
  1,
  1,
  2,
  2,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  2,
  1,
  1,
  1,
  0,
  2,
  1,
  1,
  1,
  2,
  0,
  2,
  0,
  2,
  0,
  2,
  1,
  2,
  1,
  1,
  2,
  1,
  1,
  1])